<a href="https://colab.research.google.com/github/OmdenaAI/trieste-italy-long-covid/blob/main/src/tasks/task-6-sentiment_analysis-text_classification-topic_modeling/XLM_T_ZeroShot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### test on italian

In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import pipeline
import numpy as np
from scipy.spatial.distance import cosine
from collections import defaultdict 
import urllib
import pandas as pd
from scipy.special import softmax
from sklearn.metrics import classification_report

In [ ]:
# Fetch dataset 
language = 'italian'

df = pd.read_csv('data/tweets_batch_F.csv')
df.shape, display(df.sample(3))

,Unnamed: 0,Date,User,Tweet_id,Tweet
8430,8430,2021-07-25 15:46:22+00:00,Princeps____,1419323145589252104,Due sono le cose: o abbiamo dei fascisti strao...
13513,13513,2021-08-22 07:11:15+00:00,TommyBrain,1429340373147656193,NeuroScienze:Le ombre della pandemia da COVID-...
5899,5899,2021-08-01 17:34:43+00:00,infoitinterno,1421887129546903554,"Covid19, “Basta dittatura”, il popolo del No a..."


((15317, 5), None)

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,Date,User,Tweet_id,Tweet
0,2021-09-05 04:03:23+00:00,ezakiclip,1434366525444612102,Mano…
1,2021-09-05 04:06:09+00:00,539th,1434367222668042240,Articolo su giornale neozelandese che parla di...
2,2021-09-05 04:11:14+00:00,CGTNOfficial,1434368501200736264,#COVID19 case in Guangzhou very similar to imp...


In [ ]:
# fetch XLM-T model
MODEL =  "cardiffnlp/twitter-xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.out

In [ ]:
CUDA = False
BATCH_SIZE = 32

if CUDA:
    model = model.to('cuda')

In [ ]:
# helper functions
def preprocess(corpus):
    outcorpus = []
    for text in corpus:
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        new_text = " ".join(new_text)
        outcorpus.append(new_text)
    return outcorpus

def predict(text, cuda):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', padding = True, truncation = True)
    if cuda:
        encoded_input.to('cuda')
        output = model(**encoded_input)
        scores = output[0].detach().cpu().numpy()
    else:
        output = model(**encoded_input)
        scores = output[0].detach().numpy()
  
    scores = softmax(scores, axis=-1)
    return scores

In [ ]:
#df_1 = df.sample(1000)
df_1 = df

In [ ]:
df_tweet_list = df_1['Tweet'].to_list()

In [ ]:
df_tweet_list[0:10]

['Mano…',
 'Articolo su giornale neozelandese che parla di Singapore e del suo attuale elevato numero di diagnosi pur con 80% di vaccinati ma per ora non troppi casi gravi. \nhttps://t.co/1pzPZOJtaq',
 '#COVID19 case in Guangzhou very similar to imported case https://t.co/hZovey5AAv https://t.co/4NlOFVmPNF',
 '"Certificato COVID digitale dell\'UE: | Commissione europea" https://t.co/kPLHF3DFfG. Trova le differenze con quello italiano ?#greenpass',
 'I no green pass tornano in piazza: "in giro" per Milano per ore https://t.co/kpUUt2wAtY',
 'Con sta #mascherina obbligatoria dormire in #treno è praticamente impossibile...Ma si fa, un piccolo sacrificio per sconfiggere sto dannato #COVID19',
 'Solo 20 casi in Nuova Zelanda. 15 erano contatti di casi noti ( 11 conviventi).\nSolo 6 sono stati considerati infettivi in giorni nei quali hanno avuto contatti in comunità,generando solo 6 "location of interest".',
 '#Vaccini anti #COVID19 in Italia, aggiornamento del 05/09/2021 ore 06:14\nFinora s

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(df_tweet_list, batch_size=BATCH_SIZE)
all_preds = []

In [ ]:
for idx,batch in enumerate(dl):
    if idx % 10 == 0: print('Batch ',idx+1,' of ',len(dl))
    text = preprocess(batch)
    scores = predict(text, CUDA)
    preds = np.argmax(scores, axis=-1)
    all_preds.extend(preds)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batch  1  of  479
Batch  11  of  479
Batch  21  of  479
Batch  31  of  479
Batch  41  of  479
Batch  51  of  479
Batch  61  of  479
Batch  71  of  479
Batch  81  of  479
Batch  91  of  479
Batch  101  of  479
Batch  111  of  479
Batch  121  of  479
Batch  131  of  479
Batch  141  of  479
Batch  151  of  479
Batch  161  of  479
Batch  171  of  479
Batch  181  of  479
Batch  191  of  479
Batch  201  of  479
Batch  211  of  479
Batch  221  of  479
Batch  231  of  479
Batch  241  of  479
Batch  251  of  479
Batch  261  of  479
Batch  271  of  479
Batch  281  of  479
Batch  291  of  479
Batch  301  of  479
Batch  311  of  479
Batch  321  of  479
Batch  331  of  479
Batch  341  of  479
Batch  351  of  479
Batch  361  of  479
Batch  371  of  479
Batch  381  of  479
Batch  391  of  479
Batch  401  of  479
Batch  411  of  479
Batch  421  of  479
Batch  431  of  479
Batch  441  of  479
Batch  451  of  479
Batch  461  of  479
Batch  471  of  479


In [ ]:
len(all_preds)

15317

In [ ]:
df_1['zero_shot_label'] = all_preds

In [ ]:
df_1.sample(10)

,Date,User,Tweet_id,Tweet,zero_shot_label
7833,2021-07-25 12:57:02+00:00,sbruffino,1419280533754949632,Momento autoreferenziale. \nFebbraio 2021.\n\n...,0
5855,2021-08-01 17:17:21+00:00,semplicesai,1421882760080277504,"Coronavirus in Irpinia, costante il numero di ...",0
12894,2021-08-15 17:08:29+00:00,LaLinea73167367,1426953958636011520,@MarkChierici @VincenzoDeLuca Peccato che il 9...,0
12999,2021-08-15 18:18:31+00:00,ElezioniUsa,1426971582690385929,Gli Stati Uniti potrebbero raggiungere quota 2...,0
13852,2021-08-22 10:23:50+00:00,italianelcuore3,1429388838732980231,Infermiere vaccinato con la doppia dose muore ...,0
8917,2021-07-25 18:19:58+00:00,mendeiev,1419361799892242433,Cercava di fare l'imprenditore è finito a fare...,0
3174,2021-08-29 13:52:01+00:00,zazoomblog,1431977945329963016,Coronavirus calano i positivi nel Lazio (127 a...,0
8267,2021-07-25 15:09:27+00:00,ApuntoM,1419313855793090560,#COVID19 dati ultime #24ore provincia di Trent...,0
14055,2021-08-22 12:06:40+00:00,Vvelasciavivere,1429414717743943688,"Se hai la febbre, anche se si tratta di un bam...",0
11329,2021-08-08 18:09:41+00:00,TG24info,1424432641739272200,Regione – CORONAVIRUS: nel #Lazio 554 nuovi po...,0


In [ ]:
df_1.zero_shot_label.unique()

array([0, 1])

In [ ]:
df_1.to_csv('data/tweets_batch_F.ZS_sample_result.csv')

In [ ]:
df_1['zero_shot_label'].value_counts()

0    15283
1       34
Name: zero_shot_label, dtype: int64

In [ ]:
df_1['zero_shot_label'].value_counts()

0    15283
1       34
Name: zero_shot_label, dtype: int64

In [ ]:
df_1[df_1.zero_shot_label == 1]

,Date,User,Tweet_id,Tweet,zero_shot_label
2015,2021-09-05 21:18:58+00:00,_nahuelfr,1434627136384552960,Jajsjsjsjsjsjaja q hdp,1
2171,2021-09-06 00:01:40+00:00,alexanderjos4,1434668081821986817,Jssjsjjsjsjsjsjsjs,1
2176,2021-09-06 00:09:10+00:00,Faby_Nava77,1434669971624402947,Taaaannntttiiittttaaa Maaaddre 🤬,1
3626,2021-08-29 16:23:56+00:00,PepoPeru,1432016174770966532,Jsjajsjsbsjjajzbbsbhababshhshahshhdhsh\n\n❤,1
4198,2021-08-29 23:41:36+00:00,ROBCURF,1432126318582190080,AAAAAA ZOMBIE APOCALYPSE,1
6971,2021-07-25 08:01:55+00:00,yus_AtinySpirit,1419206262332350464,HAHAHHAAHAHAHAHAHAHAHAAHAHAHAHAHAHAHAAHHAHAHAHAHA,1
6981,2021-07-25 08:07:48+00:00,atyrasabryna,1419207744612016133,SHITBDJDJSKSKKSKSDNDBDJSB,1
6982,2021-07-25 08:07:53+00:00,swattaxJournal,1419207766212632578,HAHAHAHAAHHAHAAHHAHAAHAHAH,1
7023,2021-07-25 08:23:06+00:00,najiehaks,1419211596409098240,APA DOSAKU KDJFJSJJSJDJJSCN WHYYY,1
7791,2021-07-25 12:46:26+00:00,prabha_j,1419277864474750982,@narendramodi acche din,1
